In [66]:
import asyncio
import pybotters
import pandas as pd
from loguru import logger

import pandas as pd
import datetime
import math

In [67]:
# e.g. https://api.cryptowat.ch/markets/:exchange/:pair/ohlc
def generate_path(
        exchange: str,
        pair: str,
        min: int,
        unixTime: int) -> str:
    
    return f'https://api.cryptowat.ch/markets/{exchange}/{pair}/ohlc?periods={min}&after={unixTime}'

print(generate_path('bitflyer', 'btcfxjpy', 900, 1675368565))

https://api.cryptowat.ch/markets/bitflyer/btcfxjpy/ohlc?periods=900&after=1675368565


In [68]:
def str_to_timestamp(s: str) -> int:

    # str → datetime e.g.'2022-05-27 10:11:23'
    dt = datetime.datetime.strptime(s, "%Y-%m-%d %H:%M:%S")

    # datetime → timestamp
    ts = datetime.datetime.timestamp(dt)

    return math.floor(ts)

ts = str_to_timestamp('2022-05-27 10:11:23')
print(ts)

1653613883


In [69]:
async def main(candle: int, date: str, size: int) -> pd.DataFrame:
    async with pybotters.Client() as client:
        
        url = "https://api.cryptowat.ch/markets/bitflyer/btcfxjpy/ohlc?periods="
        candle = candle*60
        period = ("%s"%(candle))
        period = [period]
        beforeurl = "&before="
        afterurl = "&after="
        # The maximum size of data is 6000 depending on a timing (sometimes you get less than 6000)
        before = str_to_timestamp(date)
        after = before - (candle * size)
        URL=("%s%s%s%s%s%s"%(url, candle, afterurl, after, beforeurl, before))
        # e.g https://api.cryptowat.ch/markets/bitflyer/btcjpy/ohlc?periods=86400&after=1483196400

        response = await client.get(URL)
        body = await response.json()
        res = body['result']
        
        data = []
        for i in period:
            row = res[i]
            for column in row:
                if column[4] != 0:
                    column = column[0:6]
                    data.append(column)

        date = [price[0] for price in data]
        op = [int(price[1]) for price in data]
        hi = [int(price[2]) for price in data]
        lo = [int(price[3]) for price in data]
        cl = [int(price[4]) for price in data]
        volume = [int(price[5]) for price in data]
        date_datetime = map(datetime.datetime.fromtimestamp, date)
        dti = pd.DatetimeIndex(date_datetime)
        df = pd.DataFrame({"op" : op, "hi" : hi, "lo": lo, "cl" : cl, "volume" : volume}, index=dti)
        
        return df

In [70]:
if __name__ == '__main__':
    try:
        df = await main(15, '2023-02-02 20:09:25', 6000)
        display(df)
        df.to_pickle('df_ohlcv.pkl')
    except KeyboardInterrupt:
        pass

,op,hi,lo,cl,volume
2022-12-03 17:15:00,2299717,2299949,2296876,2297998,64
2022-12-03 17:30:00,2298287,2299335,2296649,2298026,35
2022-12-03 17:45:00,2298046,2298289,2295326,2295604,25
2022-12-03 18:00:00,2295342,2298682,2293000,2295615,77
2022-12-03 18:15:00,2295696,2298321,2294116,2297404,30
...,...,...,...,...,...
2023-02-02 19:00:00,3076436,3080258,3074061,3079758,38
2023-02-02 19:15:00,3079612,3080461,3076000,3077042,26
2023-02-02 19:30:00,3076671,3078445,3075097,3076117,22
2023-02-02 19:45:00,3076233,3076907,3071484,3074388,27
